In [248]:
# loading necessary packages
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import os

In [249]:
# reading data
df = pd.read_csv('df_for_w2v')

In [250]:
df.head()

,Unnamed: 0,id,qid1,qid2,question1,question2,is_duplicate
0,395627,395648,494699,528639,does crow see from just one eye,how far can one see with the naked eye how is ...,0
1,25052,25054,46729,46730,how do i unblock fortiguard application control,how can i unblock torrents,0
2,44746,44749,80277,80278,how can i make money off bitcoin,can you make money with bitcoin,0
3,28221,28223,52353,52354,who is the head of immigration officer of mala...,how do avascular plants and vascular plants di...,0
4,306360,306380,429895,429896,what is ionizing radiation,can you feel ionizing radiation,0


In [251]:
df_w2v = df.drop([ 'qid1', 'qid2', 'Unnamed: 0'], axis=1)

In [252]:
df_w2v.head()

,id,question1,question2,is_duplicate
0,395648,does crow see from just one eye,how far can one see with the naked eye how is ...,0
1,25054,how do i unblock fortiguard application control,how can i unblock torrents,0
2,44749,how can i make money off bitcoin,can you make money with bitcoin,0
3,28223,who is the head of immigration officer of mala...,how do avascular plants and vascular plants di...,0
4,306380,what is ionizing radiation,can you feel ionizing radiation,0


In [233]:
from gensim.models import KeyedVectors
from gensim import corpora, models, similarities
from gensim.models import Word2Vec

w2v_model = models.KeyedVectors.load_word2vec_format('https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz', binary=True)


In [253]:
import numpy as np
from scipy import spatial 

global_index2word_set = set(w2v_model.wv.index2word)


def avg_feature_vector(sentence, model, num_features, index2word_set):
    words = sentence.split()
    feature_vec = np.zeros((num_features, ), dtype='float32')
    n_words = 0
    for word in words:
        if word in index2word_set:
            n_words += 1
            feature_vec = np.add(feature_vec, model[word])
        if (n_words > 0):
            feature_vec = np.divide(feature_vec, n_words)
    return feature_vec


/Users/Olga/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning:

Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).



In [254]:
df_1 = df.apply(lambda x: avg_feature_vector(x['question1'], model=w2v_model, num_features=300,index2word_set = global_index2word_set), axis=1)
df_2 = df.apply(lambda x: avg_feature_vector(x['question2'], model=w2v_model, num_features=300,index2word_set = global_index2word_set), axis=1)

In [255]:
df_w2v['w2v_1'] = pd.Series(df_1) 
df_w2v['w2v_2'] = pd.Series(df_2) 

In [256]:
df_w2v.head()

,id,question1,question2,is_duplicate,w2v_1,w2v_2
0,395648,does crow see from just one eye,how far can one see with the naked eye how is ...,0,"[0.024817917, 0.008890338, -0.0008907257, -0.0...","[0.005266845, -0.0010324722, 0.0058366708, 0.0..."
1,25054,how do i unblock fortiguard application control,how can i unblock torrents,0,"[0.020954385, 0.005605443, -0.018374845, -0.03...","[0.045076497, 0.019771323, 0.047888182, 0.0227..."
2,44749,how can i make money off bitcoin,can you make money with bitcoin,0,"[-0.0012916847, -0.0028290926, 0.00081380206, ...","[0.000711263, 0.0012312826, -0.0001586914, 0.0..."
3,28223,who is the head of immigration officer of mala...,how do avascular plants and vascular plants di...,0,"[-0.005887226, -0.006950822, -0.043433733, 0.0...","[-0.02272692, -0.014414852, 0.034887616, 0.042..."
4,306380,what is ionizing radiation,can you feel ionizing radiation,0,"[-0.027985891, -0.017845154, 0.1139323, -0.034...","[-0.02355957, -0.010236613, 0.07208252, -0.022..."


In [257]:
df_final = pd.DataFrame(df_w2v.w2v_1.values.tolist(), index= df_w2v.index)
df_final2 = pd.DataFrame(df_w2v.w2v_2.values.tolist(), index= df_w2v.index)

In [258]:
df_final2.head()
df_final2.shape

(100000, 300)

In [259]:
df_final['id']= df_w2v['id']
df_final2['id']= df_w2v['id']

In [260]:
# merging features
df_final  = df_final.merge(df_final2, on='id',how='left')
df_total = df_final.merge(df_w2v, on='id',how='left')

In [261]:
df_total.head()

,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,...,295_y,296_y,297_y,298_y,299_y,question1,question2,is_duplicate,w2v_1,w2v_2
0,0.024818,0.008890,-0.000891,-0.026914,-0.061265,0.043283,0.021217,-0.033485,-0.009789,0.013238,...,0.001611,0.011585,0.004289,-0.006723,0.003653,does crow see from just one eye,how far can one see with the naked eye how is ...,0,"[0.024817917, 0.008890338, -0.0008907257, -0.0...","[0.005266845, -0.0010324722, 0.0058366708, 0.0..."
1,0.020954,0.005605,-0.018375,-0.036846,-0.031907,0.000220,0.020170,-0.020317,0.003378,-0.021487,...,0.022999,-0.012044,-0.022723,-0.032795,0.083892,how do i unblock fortiguard application control,how can i unblock torrents,0,"[0.020954385, 0.005605443, -0.018374845, -0.03...","[0.045076497, 0.019771323, 0.047888182, 0.0227..."
2,-0.001292,-0.002829,0.000814,0.005542,-0.002599,-0.005087,0.000172,-0.004398,0.001510,0.006039,...,-0.001301,-0.001461,-0.001211,-0.000750,-0.003949,how can i make money off bitcoin,can you make money with bitcoin,0,"[-0.0012916847, -0.0028290926, 0.00081380206, ...","[0.000711263, 0.0012312826, -0.0001586914, 0.0..."
3,-0.001292,-0.002829,0.000814,0.005542,-0.002599,-0.005087,0.000172,-0.004398,0.001510,0.006039,...,-0.001301,-0.001461,-0.001211,-0.000750,-0.003949,how can i make money off bitcoin,can you make money with bitcoin,0,"[-0.0012916847, -0.0028290926, 0.00081380206, ...","[0.000711263, 0.0012312826, -0.0001586914, 0.0..."
4,-0.001292,-0.002829,0.000814,0.005542,-0.002599,-0.005087,0.000172,-0.004398,0.001510,0.006039,...,-0.001301,-0.001461,-0.001211,-0.000750,-0.003949,how can i make money off bitcoin,can you make money with bitcoin,0,"[-0.0012916847, -0.0028290926, 0.00081380206, ...","[0.000711263, 0.0012312826, -0.0001586914, 0.0..."


In [262]:
df_total = df_total.drop(['question1', 'question2', 'w2v_1', 'w2v_2'], axis=1)
df_total.head()

,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,...,291_y,292_y,293_y,294_y,295_y,296_y,297_y,298_y,299_y,is_duplicate
0,0.024818,0.008890,-0.000891,-0.026914,-0.061265,0.043283,0.021217,-0.033485,-0.009789,0.013238,...,0.010663,-0.011110,-0.000605,-0.020479,0.001611,0.011585,0.004289,-0.006723,0.003653,0
1,0.020954,0.005605,-0.018375,-0.036846,-0.031907,0.000220,0.020170,-0.020317,0.003378,-0.021487,...,0.016241,0.023763,-0.031710,0.008480,0.022999,-0.012044,-0.022723,-0.032795,0.083892,0
2,-0.001292,-0.002829,0.000814,0.005542,-0.002599,-0.005087,0.000172,-0.004398,0.001510,0.006039,...,0.002098,-0.007056,-0.002325,-0.003543,-0.001301,-0.001461,-0.001211,-0.000750,-0.003949,0
3,-0.001292,-0.002829,0.000814,0.005542,-0.002599,-0.005087,0.000172,-0.004398,0.001510,0.006039,...,0.002098,-0.007056,-0.002325,-0.003543,-0.001301,-0.001461,-0.001211,-0.000750,-0.003949,0
4,-0.001292,-0.002829,0.000814,0.005542,-0.002599,-0.005087,0.000172,-0.004398,0.001510,0.006039,...,0.002098,-0.007056,-0.002325,-0.003543,-0.001301,-0.001461,-0.001211,-0.000750,-0.003949,0


In [264]:
from sklearn.model_selection import train_test_split
import scipy 
from sklearn.metrics import f1_score, classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import xgboost as xgb

df_total = df_total.fillna(df_total.median()).clip(-1e11,1e11)

X = df_total.drop(['is_duplicate'],axis=1)
y = df_total['is_duplicate']
X_train,X_test,y_train,y_test = train_test_split(X, y , test_size = 0.3, random_state = 123)


models = []

models.append(("LogisticRegression",LogisticRegression()))
models.append(("XGBoost", xgb.XGBClassifier()))


results = []
names = []
for name,model in models:
    result = cross_val_score(model, X_train, y_train,  cv=3)
    names.append(name)
    results.append(result)

for i in range(len(names)):
    print(names[i],results[i].mean())

LogisticRegression 0.5696076782673515
XGBoost 0.851133252876367


In [265]:
from sklearn.metrics import roc_auc_score

# build XGboost model
xgb_model = xgb.XGBClassifier(max_depth=50, n_estimators=80, learning_rate=0.1, colsample_bytree=.7, gamma=0, reg_alpha=4, objective='binary:logistic', eta=0.3, silent=1, subsample=0.8).fit(X_train, y_train) 
# making predictions
xgb_prediction = xgb_model.predict(X_test)

# performance assessment
print(classification_report(y_test, xgb_prediction))

print(roc_auc_score(y_test, xgb_prediction))

              precision    recall  f1-score   support

           0       0.86      0.88      0.87     24176
           1       0.90      0.89      0.90     31908

    accuracy                           0.89     56084
   macro avg       0.88      0.88      0.88     56084
weighted avg       0.89      0.89      0.89     56084

0.8845245678405164
